## New Graph - Method 2

In [142]:
import os
import re
import string
import nltk
from nltk.corpus import stopwords
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from collections import defaultdict
#Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install python_louvain 
from importlib import reload
from community import community_louvain
from networkx.algorithms import community
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

In [7]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Read Data

In [184]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Electronics.json.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
data = []
with gzip.open('meta_Electronics.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

JSONDecodeError: Invalid \escape: line 1 column 2145 (char 2144)

### Create Dictionary

Dictionary for nodes

In [157]:
def clean(raw):
        
    cleaned = re.sub(r'[^\w\s\d]','',raw.lower())
    new_tokens = nltk.word_tokenize(cleaned)
    new_tokens = [r for r in new_tokens if r not in stopwords.words('english')]
    
    #lemmatize words
    lemmatizer = nltk.WordNetLemmatizer()
    new_tokens = [lemmatizer.lemmatize(t) for t in new_tokens]
        
    #remove numbers
    new_tokens = [words for words in new_tokens if not any(c.isdigit() for c in words)]
    
    #remove other punctuation/words
    new_tokens = [words for words in new_tokens if words not in ['amp','-', ';', '&','(',')' ',', '...','-','``',"''",'would',"'s","n't","--",'also',"'m","'ve"]]
    return new_tokens
    

In [158]:
reference = dict()
for item in data:
    asin = item['asin']
    if(asin not in reference):      
        reference[asin] = {'title' : clean(item['title']),
                           'raw' : item['title'],
                          'brand' : clean(item['brand'])}



In [110]:
reference

{'0011300000': {'title': ['genuine',
   'geovision',
   'channel',
   'party',
   'nvr',
   'ip',
   'software',
   'usb',
   'dongle',
   'onvif',
   'psia'],
  'raw': 'Genuine Geovision 1 Channel 3rd Party NVR IP Software with USB Dongle Onvif PSIA',
  'brand': ['geovision']},
 '0043396828': {'title': ['book',
   'handbook',
   'astronomical',
   'image',
   'processing',
   'cd',
   'rom',
   'edition',
   'hardcover',
   'book',
   'berry',
   'amp',
   'burnell'],
  'raw': 'Books "Handbook of Astronomical Image Processing" with CD ROM, 2nd Edition, Hardcover Book by Berry &amp; Burnell',
  'brand': ['book', 'co']},
 '0060009810': {'title': ['one', 'hot', 'summer'],
  'raw': 'One Hot Summer',
  'brand': ['visit', 'amazon', 'carolina', 'garcia', 'aguilera', 'page']},
 '0060219602': {'title': ['hurray',
   'hattie',
   'rabbit',
   'story',
   'picture',
   'early',
   'read',
   'book'],
  'raw': 'Hurray for Hattie Rabbit: Story and pictures (An Early I can read book)',
  'brand': [

In [103]:
#Dictionary for nodes

dict2 = {}

for item1 in data: 
    if item1['asin'] not in dict2: 
        dict2[item1['asin']] = 1
    elif item1['asin'] in dict2: 
        dict2[item1['asin']] += 1
    for item2 in item1['also_buy']:
        if item2 not in dict2: 
            dict2[item2] = 1
        elif item2 in dict2: 
            dict2[item2] += 1


In [104]:
max(dict2.values())

240

Dictionary for edges

In [105]:
#Dictionary for Edges
dict1 = {}

for item1 in data: 
    for item2 in item1['also_buy']:
        if (item1['asin'],item2) and (item2, item1['asin']) not in dict1: 
            dict1[(item1['asin'],item2)] = 1
        elif (item1['asin'], item2) in dict1: 
            dict1[(item1['asin'],item2)] += 1
        elif (item2, item1['asin']) in dict1:
            dict1[(item2,item1['asin'])] += 1
    list_of_pairs = [(item1['also_buy'][p1], item1['also_buy'][p2]) for p1 in range(len(item1['also_buy'])) for p2 in range(p1+1,len(item1['also_buy']))]
    for (item3,item4) in list_of_pairs: 
        if(item3, item4) and (item4, item3) not in dict1: 
            dict1[(item3,item4)] = 1
        elif (item3,item4) in dict1: 
            dict1[(item3,item4)] += 1
        elif (item4,item3) in dict1: 
            dict1[(item4,item3)] += 1
        

In [159]:
max(dict1.values())

30

In [131]:
print(len(dict2))

84490


In [140]:
print(len(reference))

48018


In [34]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install community

In [35]:
#Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install louvain 

### Create Graph

In [117]:
# Creating Network Graph
G = nx.Graph()
G.add_edges_from(dict1.keys())
print("number of nodes", G.number_of_nodes())
print("number of edges", G.number_of_edges())

number of nodes 42252
number of edges 2261965


In [118]:
for keys,values in dict1.items(): 
    G.edges[keys[0], keys[1]]['weight'] = values

In [ ]:
#pos = nx.spring_layout(G)
#nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40)
#nx.draw_networkx_edges(G, pos, alpha=0.5)

### Community Detection

In [119]:
partition = community_louvain.best_partition(G)

In [173]:
titles = dict()
nul_ct = 0
for i in partition:
    if(i not in reference):
        nul_ct += 1
    else:
        if(partition[i] not in titles):
            titles[partition[i]] = temp = dict()
            
        for j in reference[i]['title']:
            if(j not in titles[partition[i]]):
                temp[j] = 1
            else:
                titles[partition[i]][j] += 1
                
print(nul_ct)
print(len(titles))
print(len(partition))

36472
749
42252


In [185]:
from collections import Counter

top5 = dict()
for i in titles:
    c = Counter(titles[i])
    top5[i] = c.most_common()[:5]

top5

{0: [('image', 4),
  ('book', 2),
  ('handbook', 1),
  ('astronomical', 1),
  ('processing', 1)],
 1: [('one', 1), ('hot', 1), ('summer', 1)],
 2: [('hurray', 1),
  ('hattie', 1),
  ('rabbit', 1),
  ('story', 1),
  ('picture', 1)],
 3: [('college', 1), ('physic', 1)],
 4: [('girl', 1),
  ('onetrack', 1),
  ('mind', 1),
  ('confession', 1),
  ('seductress', 1)],
 5: [('player', 2), ('abcgoodefgreg', 1), ('usb', 1), ('music', 1), ('fm', 1)],
 6: [('last', 1), ('life', 1), ('novel', 1)],
 7: [('practical', 1), ('modern', 1), ('basketball', 1), ('edition', 1)],
 8: [('lord', 1), ('ring', 1), ('return', 1), ('king', 1), ('extended', 1)],
 9: [('linenlook', 1), ('pattern', 1), ('black', 1), ('lg', 1)],
 10: [('angel', 1), ('doll', 1), ('christmas', 1), ('story', 1)],
 11: [('scared', 1), ('money', 1)],
 12: [('never', 1), ('tell', 1)],
 13: [('kiss', 1), ('lot', 1), ('frog', 1)],
 14: [('saw', 1), ('novel', 1)],
 15: [('crow', 1), ('cant', 1), ('count', 1)],
 16: [('tokyo', 1),
  ('suckerpun

In [133]:
max(partition.values())

748

In [66]:
a = [1,2]
b = [3,4]
print(a.append(b))
print(a)
print(a.extend(b))
print(a)

None
[1, 2, [3, 4]]
None
[1, 2, [3, 4], 3, 4]
